# Importação

In [121]:
import os
import glob
import pandas as pd

from typing import List, Dict, TextIO, DefaultDict
from collections import defaultdict

# Caminho onde se localiza a pasta do equipamento

In [122]:
path_logs = r'/mnt/c/Users/mathe/Documents/AIML_logs/treinamento_aiml/logs'

# Criação do dataframe com as colunas necessárias

In [123]:
logs = pd.DataFrame(columns=['equipment_type', 'run_id', 'test_case', 'timestamp', 'baud_rate', 'slot', 'serial_port', 'tx_data', 'rx_data', 'response_time',
                            'command_protocol', 'command_name'])

# Funções

In [124]:
def get_equipment_type(path: str) -> List[str]:
    """Returns a list with equipment's types.
    
    ## Args:
        path (str): Logs path.
        
    ## Returns:
        List (str): List with equipment's types.
    """
    
    equipment_types = []
    entries = os.listdir(path)
    for entry in entries:
        equipment_types.append(entry)
    return equipment_types

In [125]:
def get_run_ids(path: str) -> Dict[str, List[str]]:
    """Returns a dict with equipment's types as keys and a list of run_ids as values.

    ## Args:
        path (str): Logs path.
        
    ## Returns:
        Dict (str, List[str]): Dict with equipment's types as keys and a list of run_ids as values.
    """
    
    run_ids = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        entries = os.listdir(f'{path_logs}/{equipment}')
        for entry in entries:
            run_ids[equipment].append(entry)
    return run_ids

In [126]:
def get_test_case(path: str) -> Dict[str, List[str]]:
    """Returns a dict with run_ids as keys and a list of test_cases names as values.

    ## Args:
        path (str): Logs path.
        
    ## Returns:
        Dict (str, List[str]): Dict with run_ids as keys and a list of test_cases names as values.
    """
    
    test_cases = {}
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            files = next(os.walk(f'{path_logs}/{equipment}/{run_id}'))[2]
            test_cases[run_id] = [file[10:-4] for file in files 
                                    if file.endswith('.txt', -4) and file != 'seed.txt']
    return test_cases

In [127]:
def open_file(path: str, eq_type: str, run_id: str, test_case: str) -> TextIO:
    """Open a test_case_*.txt file by receiving the path of the equipments, equipment type, run_id 
        and the number of the test case as arguments. Returns the file.

    ## Args:
        path (str): path where each equipment is located.
        eq_type (str): equipment_type.
        run_id (str): run_id.
        test_case (str): number of the test_case file.
        
    ## Returns:
        file (TextIO): the opened test_case file.
    """
    
    file = open(f'{path}/{eq_type}/{run_id}/test_case_{test_case}.txt', 'r', encoding='cp860')
    return file

In [128]:
def print_logs(path: str, eq_type: str, run_id: str, test_case: str) -> None:
    """Uses the open_file() to open a test_case_*.txt and print its contents.

    ## Args:
        path (str): path where each equipment is located.
        eq_type (str): equipment_type.
        run_id (str): run_id.
        test_case (str): number of the test_case file.
    """
    
    file = open_file(path, eq_type, run_id, test_case)
    [print(log_line) for log_line in file.readlines()]
    file.close()

In [129]:
def get_timestamp(path: str) -> DefaultDict[str, List[str]]:
    """Get the timestamp of every line in the test case text file that has a "TX" signal.

    ## Args:
        path (str): path where each equipment is located.
        
    ## Returns:
        timestamp (DefaultDict[str, List[str]]): a dictionary where the keys are the test cases and the values are a list of timestamps.
    """

    timestamp = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                while True:
                    log_line = file.readline()
                    if 'OPTICALTX:' in log_line or 'TX:' in log_line:
                        timestamp[test_case].append(log_line[1:24])
                    if not log_line:
                        break
                file.close()
    return timestamp

In [130]:
def get_timestamp_rx(row, path: str) -> DefaultDict[str, List[str]]:
    """Get the timestamp of every line in the test case text file that has a "RX" signal.

    ## Args:
        path (str): path where each equipment is located.
        
    ## Returns:

    """
    if row['test_case'] != '10000':
        return ''
    else:
        file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
        while True:
            log_line = file.readline()
            if 'OPTICALRX:' in log_line or 'RX:' in log_line:
                return log_line[1:24]
            if not log_line:
                break
        file.close()

In [131]:
def get_baud_rate(path: str) -> DefaultDict[str, List[str]]:
    baud_rate = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                while True:
                    log_line = file.readline()
                    if 'TX:' in log_line or 'OPTICALTX:' in log_line:
                        baud_rate[test_case].append(log_line[27:31])
                    if not log_line:
                        break
                file.close()
    return baud_rate

In [132]:
def get_slot(path: str) -> DefaultDict[str, List[str]]:
    slot = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                while True:
                    log_line = file.readline()
                    if 'TX:' in log_line or 'OPTICALTX:' in log_line:
                        slot[test_case].append(log_line[35:38].strip())
                    if not log_line:
                        break
                file.close()
    return slot

In [133]:
def get_serial_port(path: str) -> DefaultDict[str, List[str]]:
    serial_port = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                while True:
                    log_line = file.readline()
                    if 'OPTICALTX:' in log_line:
                        serial_port[test_case].append(log_line[40:49])
                    elif 'TX:' in log_line:
                        serial_port[test_case].append(log_line[40:45])
                    if not log_line:
                        break
                file.close()
    return serial_port

In [134]:
def get_tx_data(path: str):
    tx_data = defaultdict(list)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                while True:
                    log_line = file.readline()
                    if 'OPTICALTX:' in log_line:
                        tx_data[test_case].append(log_line[51:].rstrip('\n'))
                    elif 'TX:' in log_line:
                        tx_data[test_case].append(log_line[51:].rstrip('\n'))
                    if not log_line:
                        break
                file.close()
    return tx_data

In [135]:
def get_rx_data(row, path: str):
    if row['test_case'] != '10000':
        return ''
    else:    
        rx_data = ''
        file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
        while True:
            log_line = file.readline()
            if row['tx_data'] == log_line[51:].rstrip('\n'):
                while True:
                    rx_line = file.readline()
                    if 'OPTICALRX:' in rx_line or 'RX:' in rx_line:
                        if rx_line[35:38].strip() == row['slot']:
                            rx_data = rx_line[51:].rstrip('\n')
                            return rx_data
                    elif not rx_line:
                        break
                    else:
                        continue
            if not log_line:
                break
        file.close()
    return rx_data

# Aplicação das funções

In [136]:
logs['equipment_type'] = get_equipment_type(path_logs)

In [137]:
logs['run_id'] = logs['equipment_type'].map(get_run_ids(path_logs))
logs = logs.explode('run_id', ignore_index=True)

In [138]:
logs['test_case'] = logs['run_id'].map(get_test_case(path_logs))
logs = logs.explode('test_case', ignore_index=True)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,8721,220905_220107_851_8721_linux,9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1698,8721,220905_220107_851_8721_linux,9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,8721,220905_220107_851_8721_linux,9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1700,8721,220905_220107_851_8721_linux,9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
logs['timestamp'] = logs['test_case'].map(get_timestamp(path_logs))
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,"[19/08/2022 04:53:39.535, 19/08/2022 04:53:39....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10001,"[19/08/2022 04:55:17.232, 19/08/2022 04:55:17....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10002,"[18/08/2022 21:07:35.178, 18/08/2022 21:07:35....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10005,"[18/08/2022 21:02:55.145, 18/08/2022 21:02:55....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10006,"[18/08/2022 18:43:19.38 , 18/08/2022 18:43:19....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,8721,220905_220107_851_8721_linux,9995,"[19/08/2022 06:51:44.740, 19/08/2022 06:51:44....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1698,8721,220905_220107_851_8721_linux,9996,"[19/08/2022 04:19:29.994, 19/08/2022 04:19:30....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,8721,220905_220107_851_8721_linux,9997,"[19/08/2022 04:29:32.858, 19/08/2022 04:29:32....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1700,8721,220905_220107_851_8721_linux,9998,"[19/08/2022 02:40:24.306, 19/08/2022 02:40:24....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
logs['baud_rate'] = logs['test_case'].map(get_baud_rate(path_logs))
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,"[19/08/2022 04:53:39.535, 19/08/2022 04:53:39....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10001,"[19/08/2022 04:55:17.232, 19/08/2022 04:55:17....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10002,"[18/08/2022 21:07:35.178, 18/08/2022 21:07:35....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10005,"[18/08/2022 21:02:55.145, 18/08/2022 21:02:55....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10006,"[18/08/2022 18:43:19.38 , 18/08/2022 18:43:19....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,8721,220905_220107_851_8721_linux,9995,"[19/08/2022 06:51:44.740, 19/08/2022 06:51:44....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1698,8721,220905_220107_851_8721_linux,9996,"[19/08/2022 04:19:29.994, 19/08/2022 04:19:30....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,8721,220905_220107_851_8721_linux,9997,"[19/08/2022 04:29:32.858, 19/08/2022 04:29:32....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1700,8721,220905_220107_851_8721_linux,9998,"[19/08/2022 02:40:24.306, 19/08/2022 02:40:24....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
logs['slot'] = logs['test_case'].map(get_slot(path_logs))
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,"[19/08/2022 04:53:39.535, 19/08/2022 04:53:39....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, OPT, OPT, OPT, O...",NaN,NaN,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10001,"[19/08/2022 04:55:17.232, 19/08/2022 04:55:17....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, OPT, OPT, OPT, OPT, B, B, B...",NaN,NaN,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10002,"[18/08/2022 21:07:35.178, 18/08/2022 21:07:35....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, OPT, OPT, OPT, OPT, OPT, OP...",NaN,NaN,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10005,"[18/08/2022 21:02:55.145, 18/08/2022 21:02:55....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, OPT, OPT, OPT, OPT, O...",NaN,NaN,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10006,"[18/08/2022 18:43:19.38 , 18/08/2022 18:43:19....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, B, B, B, OPT, OPT, OPT, OP...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,8721,220905_220107_851_8721_linux,9995,"[19/08/2022 06:51:44.740, 19/08/2022 06:51:44....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, OPT, OPT, OPT, O...",NaN,NaN,NaN,NaN,NaN,NaN
1698,8721,220905_220107_851_8721_linux,9996,"[19/08/2022 04:19:29.994, 19/08/2022 04:19:30....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, B, B, OPT, OPT, OPT, ...",NaN,NaN,NaN,NaN,NaN,NaN
1699,8721,220905_220107_851_8721_linux,9997,"[19/08/2022 04:29:32.858, 19/08/2022 04:29:32....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, B, B, B, OPT, OP...",NaN,NaN,NaN,NaN,NaN,NaN
1700,8721,220905_220107_851_8721_linux,9998,"[19/08/2022 02:40:24.306, 19/08/2022 02:40:24....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, OPT, OPT, OPT, OPT, O...",NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
logs['serial_port'] = logs['test_case'].map(get_serial_port(path_logs))
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,"[19/08/2022 04:53:39.535, 19/08/2022 04:53:39....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM20, COM20, COM20, COM...",NaN,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10001,"[19/08/2022 04:55:17.232, 19/08/2022 04:55:17....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, OPT, OPT, OPT, OPT, B, B, B...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...",NaN,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10002,"[18/08/2022 21:07:35.178, 18/08/2022 21:07:35....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, OPT, OPT, OPT, OPT, OPT, OP...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...",NaN,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10005,"[18/08/2022 21:02:55.145, 18/08/2022 21:02:55....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...",NaN,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10006,"[18/08/2022 18:43:19.38 , 18/08/2022 18:43:19....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, B, B, B, OPT, OPT, OPT, OP...","[COM22, COM22, COM22, COM20, COM20, SLOTB, SLO...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,8721,220905_220107_851_8721_linux,9995,"[19/08/2022 06:51:44.740, 19/08/2022 06:51:44....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM20, COM20, COM20, COM...",NaN,NaN,NaN,NaN,NaN
1698,8721,220905_220107_851_8721_linux,9996,"[19/08/2022 04:19:29.994, 19/08/2022 04:19:30....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, B, B, OPT, OPT, OPT, ...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...",NaN,NaN,NaN,NaN,NaN
1699,8721,220905_220107_851_8721_linux,9997,"[19/08/2022 04:29:32.858, 19/08/2022 04:29:32....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, B, B, B, OPT, OP...","[COM22, COM22, COM22, COM20, COM20, COM20, COM...",NaN,NaN,NaN,NaN,NaN
1700,8721,220905_220107_851_8721_linux,9998,"[19/08/2022 02:40:24.306, 19/08/2022 02:40:24....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...",NaN,NaN,NaN,NaN,NaN


In [143]:
logs['tx_data'] = logs['test_case'].map(get_tx_data(path_logs))
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,"[19/08/2022 04:53:39.535, 19/08/2022 04:53:39....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM20, COM20, COM20, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10001,"[19/08/2022 04:55:17.232, 19/08/2022 04:55:17....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, OPT, OPT, OPT, OPT, B, B, B...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10002,"[18/08/2022 21:07:35.178, 18/08/2022 21:07:35....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, OPT, OPT, OPT, OPT, OPT, OP...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10005,"[18/08/2022 21:02:55.145, 18/08/2022 21:02:55....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10006,"[18/08/2022 18:43:19.38 , 18/08/2022 18:43:19....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, B, B, B, OPT, OPT, OPT, OP...","[COM22, COM22, COM22, COM20, COM20, SLOTB, SLO...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1697,8721,220905_220107_851_8721_linux,9995,"[19/08/2022 06:51:44.740, 19/08/2022 06:51:44....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM20, COM20, COM20, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
1698,8721,220905_220107_851_8721_linux,9996,"[19/08/2022 04:19:29.994, 19/08/2022 04:19:30....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, B, B, OPT, OPT, OPT, ...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
1699,8721,220905_220107_851_8721_linux,9997,"[19/08/2022 04:29:32.858, 19/08/2022 04:29:32....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, OPT, OPT, OPT, OPT, B, B, B, OPT, OP...","[COM22, COM22, COM22, COM20, COM20, COM20, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN
1700,8721,220905_220107_851_8721_linux,9998,"[19/08/2022 02:40:24.306, 19/08/2022 02:40:24....","[9600, 9600, 9600, 9600, 9600, 9600, 9600, 960...","[B, B, B, B, B, B, B, B, OPT, OPT, OPT, OPT, O...","[COM22, COM22, COM22, COM22, COM22, COM22, COM...","[02 01 07 00 09 0C 03, 02 01 07 00 00 05 03, 0...",NaN,NaN,NaN,NaN


In [144]:
logs = logs.explode(['timestamp', 'baud_rate', 'slot', 'serial_port', 'tx_data'], ignore_index=True)
logs


,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,NaN,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,NaN,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,NaN,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
324049,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,NaN
324050,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,NaN
324051,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,NaN
324052,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,NaN


In [146]:
logs['rx_data'] = logs.apply(get_rx_data, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...,NaN,NaN,NaN
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...,NaN,NaN,NaN
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...,NaN,NaN,NaN
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...,NaN,NaN,NaN
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
324049,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN
324050,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN
324051,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN
324052,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN


In [147]:
logs['timestamp_rx'] = logs.apply(get_timestamp_rx, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name,timestamp_rx
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324049,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN,
324050,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN,
324051,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN,
324052,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,,NaN,NaN,NaN,


In [150]:
logs.query('test_case == "10000"')

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,response_time,command_protocol,command_name,timestamp_rx
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
5,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:41.41,9600,OPT,COM20,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,21 12 34 56 78 09 21 35 27 02 12 02 09 21 00 2...,NaN,NaN,NaN,19/08/2022 04:53:39.564
6,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:41.407,9600,OPT,COM20,63 12 34 56 00 01 00 00 00 00 00 00 00 00 00 0...,63 12 34 56 78 00 01 00 00 00 00 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
7,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:41.776,9600,OPT,COM20,E2 12 34 56 29 01 00 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 29 01 00 01 FF FF FF FF FF FF F...,NaN,NaN,NaN,19/08/2022 04:53:39.564
8,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:42.143,9600,OPT,COM20,E2 12 34 56 2F 00 00 00 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 00 01 01 01 01 00 00 00 00 0...,NaN,NaN,NaN,19/08/2022 04:53:39.564
9,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:42.507,9600,OPT,COM20,21 12 34 56 00 02 00 00 00 00 00 00 00 00 00 0...,21 12 34 56 78 09 21 37 27 02 12 02 09 21 00 2...,NaN,NaN,NaN,19/08/2022 04:53:39.564
